## User Management
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import utils.DemoClient as client
import jwt
import json

## Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "185.52.193.87.nip.io"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

### Authenticate

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="UserA"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)
print("user_id_token:", user_id_token)

### Inspect the ID Token

In [ ]:
# Inspect ID Token
jwt_header = jwt.get_unverified_header(user_id_token)
jwt_payload = jwt.decode(user_id_token, options={"verify_signature": False})
print("JWT Header:", json.dumps(jwt_header, indent = 2), "\n---\nJWT Payload:", json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Access Protected Endpoint
We perform an example access to an ADES resource.
The ADES is protected by a PEP (Policy Enforcement Point). When accessing the protected endpoint the client follows a UMA (User Managed Access) flow to obtain authorized access on behalf of the user. The UMA flow utilises the ID Token to obtain a 'ticket' which can then be exchanged by the client for an access token (RPT - Relying Party Token) that acts as a short-lived (e.g. 5 mins) credential for the specific access by a specific client, to a specific resource, on behalf of a specific user.

In [ ]:
# Call ADES 'List Processes' endpoint
ades_proc_url = "http://ades." + platform_domain + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

trace = demo.trace_flow; demo.trace_flow = True
response, process_ids, access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token)
demo.trace_flow = trace

In [ ]:
# List of processes returned...
print("Processes:", process_ids)